<a href="https://colab.research.google.com/github/marcosbenicio/deep_learning_classification/blob/main/classificador_05_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador para os dígitos $0$ e $5$

Instalação manual de pacote

In [8]:
!pip install -Uqq fastbook

Biblioteca fastai permite usar funções para rapidamente construir uma rede neuronal e treinar nosso modelo de diferenciar $0$ e $5$.

In [9]:
import fastbook
from fastai.vision.all import *
fastbook.setup_book()
from fastbook import *
import numpy as np

# Muda a escala de cor padrão da imagem para escala de cinza.
matplotlib.rc('image', cmap='Greys')

import PIL as pl
import torch as t
import pandas as pd

Primeiro passamos o caminho do diretório dataset MINST para a variável path. Nesse diretório temos imagens para todos os dígitos de $0$ à $9$. Em seguida, com o código (/path/...).ls( ), é possível listar o conteúdo dentro desse diretório. Temos portanto a pasta 'training' e 'testing', e dentro de cada um delas mais outros $10$ diretórios referentes aos números de $0$ à $9$. Esse método retorna uma lista contendo os caminhos dos arquivos de imagem, cada posição da lista têm armazenado o caminho para um arquivo de imagem.

Por ultimo, nessa célula, vamos criar quatro listas com os caminhos dos arquivos em ordem crescente. Teremos duas listas contendo todas os caminhos das imagens de $0$s(zeros) presentes nos diretórios 'training' e 'testing', e outros duas listas para as imagens de $5$s(cincos). 

In [10]:
# Armazena o caminho para o diretório do dataset MNIST numa variável.
path = untar_data(URLs.MNIST)

# Para o diretório 'training' 
zeros_train = (path/'training'/'0').ls().sorted()
cincos_train = (path/'training'/'5').ls().sorted()

# Para o diretório 'testing' 
zeros_test = (path/'testing'/'0').ls().sorted()
cincos_test = (path/'testing'/'5').ls().sorted()

Criarei quatro listas contendo tensores de segunda ordem (matrizes)  de dimensão $28\times28$ representando cada uma das imagens. Portanto ficarei com duas listas contendo todas as imagens de $0$ dos diretórios 'training' e 'testing', e outras duas listas para as imagens de $5$. 

Além disso, vamos empacotar cada lista num tensor de primeira ordem (vetor) usando o método stack da biblioteca Pytorch. Irei também normalizar o tensor para os elementos do ultimo eixo assumirem valores entre $0$ e $1$.
Para o diretório de zeros, por exemplo, o tensor que definiremos como zeros_train_tensor terá $5923$ matrizes de dimensão $28\times28$.

 

In [11]:
# Para o diretório 'training'
zeros_train_list = [tensor(Image.open(i)) for i in zeros_train]
cincos_train_list = [tensor(Image.open(i)) for i in cincos_train]

zeros_train_tensor = t.stack(zeros_train_list).float()/255
cincos_train_tensor = t.stack(cincos_train_list).float()/255

# Para o diretório 'testing'
zeros_test_list = [tensor(Image.open(i)) for i in zeros_test]
cincos_test_list = [tensor(Image.open(i)) for i in cincos_test]

zeros_test_tensor = t.stack(zeros_test_list).float()/255
cincos_test_tensor = t.stack(cincos_test_list).float()/255


Nossa variável independente $x$ serão as imagens $0$s e $5$s do diretório 'training' concatenanas num único tensor x_train. Para isso usarei o método view, que  muda o formato do tensor sem alterar seu conteúdo. O $-1$ é um parametro especial que pega o tamanho necessário de um eixo para caber todos os dados.

O tensor y_train armazenará o rótulo referente a cada imagem do tensor x_train. O método unsqueeze(1) adiciona uma dimensão ao tensor, ficando equiparável ao rank do tensor x_train. O mesmo processo vai ser feito também para o diretório 'testing', que chamarei de x_test e y_test.

Além disso, usando a função zip podemos colocar elementos de mesmo indice e diferentes tensores como elementos de uma mesma tupla (x,y). Portanto, relacionariamos cada imagem de x_test com um rótulo de y_test através das tuplas (x,y) como um conjunto de pontos de um gráfico.



In [12]:
# Para o diretório 'training'
x_train = t.cat([zeros_train_tensor, cincos_train_tensor]).view(-1, 28*28)
y_train = tensor([0]*len(zeros_train) + [1]*len(cincos_train)).unsqueeze(1)

dset_train = list(zip(x_train, y_train))

# Para o diretório 'testing'
x_test = t.cat([zeros_test_tensor, cincos_test_tensor]).view(-1, 28*28)
y_test = tensor([0]*len(zeros_test) + [1]*len(cincos_test)).unsqueeze(1)

dset_test = list(zip(x_test, y_test))

Vamos inicializar um tensor usando o método randn(size) dentro da função inicia_parametros( )  que vamos definir. Essa função recebe o número total de pixels presentes nas imagens, que no nosso caso são $28 \times 28=784$ pixels.
Usando essa função teremos dois tensores, weight (peso) e bias. .



In [13]:
def inicia_parametros(num_pixel): return (t.randn(num_pixel))

weights = inicia_parametros((28*28,1))
bias = inicia_parametros(1)

Escrevemos uma função do tipo:
$$ \vec{y} = {X}.\vec{w} +b$$
onde $\vec{Y}$ é um vetor, $X$ um tensor e $b$ um escalar. A ideia é evitr o uso de loops, trocando por um produto entre tensores. Isso fornecerá y_previsao, que serão os valores que melhor se aproximam de y_train.

In [15]:
def linear1(xb): return xb@weights + bias
y_previsao = linear1(x_train)

#print(y_previsao)
#print(y_train)
#print(weights[:4])
#print(bias)
# previsao seriam os valores de y proximos do valor ideal.



Queremos comparar os tensores y_previsao e y_train de rank-1.


Usando operadores booleanos vamos crir um tensor com variáveis 0(false), se o valor for  menor que $0.5$ e 1(true) no caso contrário. Note que mesmo sendo um tensor de zeros e uns ele é definido como float. Em seguida usando o condicional $==$ vamos comparar com os valores presentes no tensor y_train. Lembre que y_train é o tensor de rank-1 contendo  os rótulos $0$ e $1$ das imagens presentes no tensor x_train.

In [16]:
correta = (y_previsao > 0.5).float() == y_train
data_correta.float().mean().item()


NameError: ignored

A função t.where(a,b,c) vai medir a distancia do valor da previsão com relação a 1 e quão distante é de zero para em seguir tirar a média de todas as distancias.

In [ ]:
def mnist_loss(y_previsao, alvo):
    return t.where(alvos==1, 1-y_previsao, previsao).mean()

    

In [ ]:
dl_train = DataLoader(dset_train, batch_size=256)
dl_test = DataLoader(dset_test, batch_size=256)

Separarei os dados de x_train em batchs, como se fossem pequenos pacotes

In [ ]:
batch = x_train[0:4]
previsa = linear1(batch)

loss = mnist_loss(previsa, y_train[:4])
